Trying [lambda cloud](cloud.lambda.ai)

Made an account and added credit card and billing info.

Want to try first with a ~$0.50 / hour one like I was using on paperspace but those all show out of capacity.

What's the difference between SXM5 and PCIe? I'll go with 1x H100 80 GB SXM5.

Only 2 regions available. Going with us-south-2

Unsure which base image to use. Will go with default: Lambda Stack 22.04

Filesystem seems much more expensive than paperspace...or am I calculating wrong?

Created filesystem: mynanochat

Did not create firewall rules

Booting...said it could take 10 minutes. Tooks less. Maybe 2 miutes?

Going to paste the instructions for setting up a paperspace machine here and edit as I go.

### Instructions

Consolidating instructions here so I don't have to look back at challenge 14.

```
ssh ssh ubuntu@[ip]

# ssh key for git
ssh-keygen -t ed25519 -C "lambda-cloud"
cat ~/.ssh/id_ed25519.pub
copy into github UI (https://github.com/settings/keys)

git config --global user.email "ericsilberstein@gmail.com"
git config --global user.name "Eric Silberstein"

# clone this repo
git clone git@github.com:ericsilberstein1/nanogpt-learning.git

# Basic packages
# sudo apt-get update && sudo apt-get install -y build-essential git curl wget ca-certificates

# confirm / install smi
#nvidia-detector
#nvidia-smi
#sudo apt-get install nvidia-driver-580 nvidia-utils-580
#sudo modprobe nvidia # (before did sudo shutdown -r now)
#nvidia-smi

# later, after working on challenge 15, looked into why torch was failing to compile the model
# and realized needed to do this so python headers get installed
# the specific error was: /tmp/tmp0normshd/cuda_utils.c:6:10: fatal error: Python.h: No such file 
# or directory 6 | #include <Python.h>
#sudo apt-get install python3.10-dev

# UV
curl -LsSf https://astral.sh/uv/install.sh | sh
#source .zshrc

# rust
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
echo '. "$HOME/.cargo/env"' >> .bashrc
source .bashrc

cd nanogpt-learning

uv sync
source .venv/bin/activate

# download data files
PYTHONPATH=my_nanochat python -m my_nanochat.my_dataset --num-files 8

# download another 240 in the background, check this completes before starting base_train
PYTHONPATH=my_nanochat python -m my_nanochat.my_dataset --num-files 240 &

# for now until organize this better
uv tool install maturin
cd challenge-07-rust-and-python-simplified-tokenizer/rust_tokenizer
maturin develop
cd -

# in challenge 23 I created my_tok_train.py so now can train the tokenizer here rather
# than from a notebook
PYTHONPATH=my_nanochat python -m scripts.my_tok_train

# while that's running

# in .zshrc add
# export WANDB_API_KEY="XXX"

uv run jupyter lab

# ON MY LAPTOP make port 8889 a tunnel to jupyter
ssh -N -L 8889:localhost:8888 paperspace@[public_ip]
```

Make base dir configurable.

Quick try, on my mac still:

In [1]:
!pwd

/Users/ericsilberstein/Documents/ericsilberstein1-repos/nanogpt-learning/challenge-25-pretrain-d20


In [2]:
import sys
sys.path.append('../my_nanochat')
from my_nanochat.my_common import get_base_dir

In [3]:
get_base_dir()

'/Users/ericsilberstein/.cache/my_nanochat'

In [10]:
import os
os.environ['NANOCHAT_BASE_DIR'] = os.getcwd()
get_base_dir()

'/Users/ericsilberstein/Documents/ericsilberstein1-repos/nanogpt-learning/challenge-25-pretrain-d20'